In [1]:
import os
#  import time
#  from multiprocess import Pool
import numpy as np
import iris
import iris.coord_categorisation as icc
#  from scipy import stats
#  import dask as da
from datetime import datetime
import joblib
from scipy import stats

In [2]:
#  specify paths
out_script = '/home/bschmidt/scripts/detrending/output/regr.out'
source_path_data = '/home/bschmidt/data/test_data_tas.nc4'
source_path_gmt = '/home/bschmidt/data/test_gmt.nc4'
dest_path = '/home/bschmidt/data/'

#  Get jobs starting time
STIME = datetime.now()
with open(out_script, 'w') as out:
    out.write('Job started at: ' + str(STIME) + '\n')
print('Job started at: ' + str(STIME))

#  load data
gmt = iris.load_cube(source_path_gmt)
#  icc.add_day_of_year(gmt, 'time')
#  print(gmt)
data = iris.load_cube(source_path_data)
icc.add_day_of_year(data, 'time')
#  print(data)

#  Get dayofyear-vectors of gmt and data
doys_cube = data.coord('day_of_year').points
#  print(doys_cube)
doys = gmt.coord('day_of_year').points
#print('Days of Year are:\n')
#print(doys)

#  Create numpy arrays as containers for regression output
print('Creating Container for Slopes')
slope = np.ones((366,
                  data.coord('latitude').shape[0],
                  data.coord('longitude').shape[0]), dtype=np.float64)
intercept=slope
r_value=slope
p_value=slope
std_err=slope

Job started at: 2019-03-07 11:45:12.957829
Creating Container for Slopes


In [3]:
def regr_doy(doy):
    # loop over lon and lat and calculate regression
    # write that into container
    gmt_day = gmt[doys == doy].data
    #A = np.vstack([gmt_day, np.ones(len(gmt_day))]).T
    print('\nDayofYear is:\n' + str(doy))
    for yx_slice in data.slices(['day_of_year']):
        #slope, intercept = np.linalg.lstsq(A, yx_slice[doys == doy].data, rcond=None)[0]
        s, i, r, p, sd = stats.linregress(gmt_day[1:-1],yx_slice[doys == doy].data[1:-1])
        #print('\nSlope is:\n')
        #print(slope)
        #print('\nIntercept is:\n')
        #print(intercept)
        #print('\nDayofYear is:\n')
        #print(doy)
        lat = int(np.where(data.coord('latitude').points == yx_slice.coord('latitude').points)[0])
        #print('\nLatitude is:\n')
        #print(lat)
        lon = int(np.where(data.coord('longitude').points == yx_slice.coord('longitude').points)[0])
        #print('\nLongitude is:\n')
        #print(lon)
        #  write regression output to containers
        slope[doy-1, lat, lon] = s
        intercept[doy-1, lat, lon] = i
        r_value[doy-1, lat, lon] = r
        p_value[doy-1, lat, lon] = p
        std_err[doy-1, lat, lon] = sd

In [4]:
from joblib import dump, load
import shutil

#  Get jobs starting time
STIME = datetime.now()

folder = './joblib_memmap'
try:
    os.mkdir(folder)
except FileExistsError:
    pass

memmap = list()
for obj in ['slope', 'intercept', 'r_value', 'p_value', 'std_err']:
    memmap.append(os.path.join(folder, obj + '_memmap'))
    dump(eval(obj), memmap[-1])
    #slopes_memmap = np.memmap(slopes_memmap, dtype=slopes.dtype, shape=slopes.shape, mode='w+')

slope = load(memmap[0], mmap_mode='r+')
intercept = load(memmap[1], mmap_mode='r+')
r_value = load(memmap[2], mmap_mode='r+')
p_value = load(memmap[3], mmap_mode='r+')
std_err = load(memmap[4], mmap_mode='r+')

# loop over dayofyear-vector, then 
print('Start with regression Calculations\n')

joblib.Parallel(n_jobs=4)(
    joblib.delayed(regr_doy)(doy) for doy in np.unique(doys))

FTIME = datetime.now()#
duration = FTIME - STIME
print('Time elapsed ' + str(duration.total_seconds()) + ' seconds!')


try:
    shutil.rmtree(folder)
except:  # noqa
    print('Could not clean-up automatically.')

Start with regression Calculations

Time elapsed 679.897469 seconds!


In [5]:
#  Create dayofyear coordinate
doy_coord = iris.coords.DimCoord(range(1,367))
for obj in ['slope', 'intercept', 'r_value', 'p_value', 'std_err']:
    dest_path_obj = os.path.join(dest_path, 'test_' + data.name() + '_' + obj + '.nc4')
    #  wrap iris cube container around data
    cube = iris.cube.Cube(eval(obj),
                        dim_coords_and_dims=[(doy_coord, 0),
                                             (data.coord('latitude'), 1),
                                             (data.coord('longitude'), 2),
                                            ])
    iris.fileformats.netcdf.save(cube, dest_path_obj)

# Get jobs finishing time
FTIME = datetime.now()
with open(out_script, 'a') as out:
    out.write('Job finished at: ' + str(FTIME) + '\n')
print('Job finished at: ' + str(FTIME))
duration = FTIME - STIME
print('Time elapsed ' +
      str(divmod(duration.total_seconds(), 3600)[0]) +
      ' hours!')

Job finished at: 2019-03-07 11:56:41.931396
Time elapsed 0.0 hours!
